In [18]:
import pandas as pd
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import skorch

In [6]:
data_ICICI_min= pd.read_csv(r"C:\Users\rgs88\OneDrive\Documents\Semester 2 Data Science Masters\Neural Computing\NN Final Project\ICICI_min_test.csv")

In [11]:
data_ICICI_min = data_ICICI_min.drop(columns = 'Unnamed: 0')

In [13]:
data_ICICI_min

,open,high,low,close
0,0.209937,0.210460,0.207598,0.206675
1,0.206229,0.206339,0.206649,0.206091
2,0.205996,0.206158,0.206714,0.206377
3,0.206579,0.206339,0.206766,0.205909
4,0.205931,0.205691,0.205713,0.204845
...,...,...,...,...
671085,0.990210,0.989504,0.992850,0.990722
671086,0.990146,0.991059,0.993305,0.992020
671087,0.991313,0.991512,0.994280,0.992279
671088,0.992220,0.992938,0.995450,0.994290


In [16]:
from sklearn.model_selection import train_test_split

def split_data(data, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["close"]), data["close"], test_size=test_ratio, random_state=random_state)

    # Split the remaining data into train and validation sets
    remaining_ratio = 1 - test_ratio
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/remaining_ratio, random_state=random_state)

    # Convert the output to pandas DataFrame
    X_train = pd.DataFrame(X_train, columns=data.drop(columns=["close"]).columns)
    y_train = pd.DataFrame(y_train, columns=["close"])
    X_val = pd.DataFrame(X_val, columns=data.drop(columns=["close"]).columns)
    y_val = pd.DataFrame(y_val, columns=["close"])
    X_test = pd.DataFrame(X_test, columns=data.drop(columns=["close"]).columns)
    y_test = pd.DataFrame(y_test, columns=["close"])

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
X_train_1, y_train_1, X_val_1, y_val_1, X_test_1, y_test_1 = split_data(ICICI_min)

In [ ]:
import torch

def convert_to_tensors(X_train, y_train, X_val, y_val, X_test, y_test):
    X_train_np = X_train.values
    X_val_np = X_val.values
    X_test_np = X_test.values
    y_train_np = y_train.values
    y_val_np = y_val.values
    y_test_np = y_test.values

    X_train_t = torch.from_numpy(X_train_np).type(torch.Tensor)
    X_val_t = torch.from_numpy(X_val_np).type(torch.Tensor)
    X_test_t = torch.from_numpy(X_test_np).type(torch.Tensor)
    y_train_t = torch.from_numpy(y_train_np).type(torch.Tensor)
    y_val_t = torch.from_numpy(y_val_np).type(torch.Tensor)
    y_test_t = torch.from_numpy(y_test_np).type(torch.Tensor)

    return X_train_t, y_train_t, X_val_t, y_val_t, X_test_t, y_test_t

In [ ]:
X_train_t_1, y_train_t_1, X_val_t_1, y_val_t_1, X_test_t_1, y_test_t_1 = convert_to_tensors(X_train_1, y_train_1, X_val_1, y_val_1, X_test_1, y_test_1)

# Models

In [17]:
rnn_hyperparams = {
    'input_size': 3,
    'hidden_size': 128,
    'num_hidden_layers': 6,
    'batch_first': True,
    'dropout': 0.5,
    'bidirectional': False,
    'output_size': 1,
    'activation_function': nn.ReLU,
    'num_epochs': 10,
    'learning_rate': 0.5,
    'momentum': 3.0,
    'batch_size': 2000
}

NameError: name 'nn' is not defined

In [ ]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, rnn_hyperparams):
        super(RNN, self).__init__()

        # Define the recurrent layers
        self.rnn = nn.ModuleList()
        self.rnn.append(nn.RNN(input_size=rnn_hyperparams['input_size'], 
                          hidden_size=rnn_hyperparams['hidden_size'],
                          batch_first=rnn_hyperparams['batch_first'],
                          dropout=rnn_hyperparams['dropout'],
                          bidirectional=rnn_hyperparams['bidirectional']))
        for i in range(1, rnn_hyperparams['num_hidden_layers']):
            self.rnn.append(nn.RNN(input_size=rnn_hyperparams['hidden_size'], 
                          hidden_size=rnn_hyperparams['hidden_size'],
                          batch_first=rnn_hyperparams['batch_first'],
                          dropout=rnn_hyperparams['dropout'],
                          bidirectional=rnn_hyperparams['bidirectional']))

        # Define any additional layers, such as a fully connected output layer
        self.fc = nn.Linear(rnn_hyperparams['hidden_size'], rnn_hyperparams['output_size'])
        
        # Saving some additional hyperparameters
        self.activation_function = rnn_hyperparams['activation_function']
        self.num_epochs = rnn_hyperparams['num_epochs']
        self.learning_rate = rnn_hyperparams['learning_rate']
        self.momentum = rnn_hyperparams['momentum']
        self.batch_size = rnn_hyperparams['batch_size']
    
    def forward(self, x):
        # Pass the input through the RNN
        for rnn_layer in self.rnn:
            x, hidden = rnn_layer(x)
        
        # Pass the final hidden state through a fully connected output layer
        out = self.fc(hidden[-1])
        
        # Apply the activation function
        out = self.activation_function(out)
        
        return out


In [ ]:
cnn_hyperparams = {
    'num_epochs': 10,
    'batch_size': 2000,
    'learning_rate': 0.5,
    'momentum': 1.0,
    'weight_decay': 0.0001,
    'activation_function' : nn.ELU,
    'num_hidden_layers': 4,
    'dropout': 0.5,
    'conv_layers': [
        (32, 3, 1, 1),   # (out_channels, kernel_size, stride, padding)
        (64, 3, 1, 1),
        (128, 3, 1, 1)
    ],
    'fc_layers': [
        (512, True),    # (out_features, use_relu)
        (256, True),
        (128, True),
        (10, False)
    ]
}

In [ ]:
class CNN(nn.Module):
    def __init__(self, cnn_hyperparams):
        super(CNN, self).__init__()

        # Extract hyperparameters
        num_epochs = cnn_hyperparams['num_epochs']
        batch_size = cnn_hyperparams['batch_size']
        learning_rate = cnn_hyperparams['learning_rate']
        momentum = cnn_hyperparams['momentum']
        weight_decay = cnn_hyperparams['weight_decay']
        dropout = cnn_hyperparams['dropout']
        conv_layers = cnn_hyperparams['conv_layers']
        fc_layers = cnn_hyperparams['fc_layers']
        num_hidden_layers = cnn_hyperparams['num_hidden_layers']
        activation_function = cnn_hyperparams['activation_function']

        # Define convolutional layers
        in_channels = 3
        conv_modules = []
        for out_channels, kernel_size, stride, padding in conv_layers:
            conv_modules.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding))
            conv_modules.append(activation_function(inplace=True))
            conv_modules.append(nn.BatchNorm2d(out_channels))
            conv_modules.append(nn.Dropout2d(p=dropout))
            in_channels = out_channels
        self.conv = nn.Sequential(*conv_modules)

        # Define fully connected layers
        fc_modules = []
        in_features = 128 * 32 * 32
        for out_features, use_relu in fc_layers:
            fc_modules.append(nn.Linear(in_features, out_features))
            if use_relu:
                fc_modules.append(activation_function(inplace=True))
                fc_modules.append(nn.BatchNorm1d(out_features))
                fc_modules.append(nn.Dropout(p=dropout))
            in_features = out_features
        self.fc = nn.Sequential(*fc_modules)

        # Define output layer
        self.output = nn.Linear(in_features, 10)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.output(x)
        return x

In [ ]:
lstm_hyperparams = {
    'input_size': 3,          # number of expected features in the input
    'hidden_size': 256,         # number of features in the hidden state
    'num_layers': 2,            # number of recurrent layers
    'bidirectional': True,      # if True, becomes a bidirectional LSTM
    'dropout': 0.5,             # dropout probability for the LSTM layers
    'batch_first': True,        # if True, expects input tensors to have shape (batch_size, seq_length, input_size)
    'batch_size': 2000,           # mini-batch size for training
    'seq_length': 20,           # length of input sequences
    'learning_rate': 0.001,     # learning rate for the optimizer
    'num_epochs': 10,           # number of epochs to train for
    'num_classes': 10,          # number of classes in the output
    'momentum': 0.9,
    'activation_function': nn.ReLU,
    'num_hidden_layers': [15,30]
}

In [ ]:
class LSTM(nn.Module):
    
    def __init__(self, lstm_hyperparams):
        super(LSTM, self).__init__()
        self.num_classes = lstm_hyperparams['num_classes'] # number of classes
        self.num_layers = lstm_hyperparams['num_layers'] # number of layers
        self.input_size = lstm_hyperparams['input_size'] # input size
        self.hidden_size = lstm_hyperparams['hidden_size'] # hidden state
        self.seq_length = lstm_hyperparams['seq_length'] # sequence length
        self.num_hidden_layers = lstm_hyperparams['num_hidden_layers'] # number of hidden layers
        
        # LSTM takes inputs, and outputs the hidden states with the dimensionality hidden_dim
        self.lstm = nn.LSTM(input_size = lstm_hyperparams['input_size'],
                            hidden_size = lstm_hyperparams['hidden_size'],
                            num_layers = lstm_hyperparams['num_layers'],
                            bidirectional=lstm_hyperparams['bidirectional'],
                            dropout = lstm_hyperparams['dropout'],
                            batch_first = lstm_hyperparams['batch_first'])
        
        self.hidden_layers = nn.ModuleList()
        for i in range(self.num_hidden_layers):
            self.hidden_layers.append(nn.Linear(lstm_hyperparams['hidden_size'], lstm_hyperparams['hidden_size']))
        
        self.fc = nn.Linear(lstm_hyperparams['hidden_size'], lstm_hyperparams['num_classes'])
        
        self.activation_function = lstm_hyperparams['activation_function']()
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers * 2 if lstm_hyperparams['bidirectional'] else self.num_layers,
                                   x.size(0),
                                   self.hidden_size)) # hidden state
        c_0 = Variable(torch.zeros(self.num_layers * 2 if lstm_hyperparams['bidirectional'] else self.num_layers,
                                   x.size(0),
                                   self.hidden_size)) # internal state
        
        # Propogate input through the LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # LSTM with input, hidden, and internal state
        
        for i in range(self.num_hidden_layers):
            output = self.hidden_layers[i](output)
            output = self.activation_function(output)
        
        out = self.fc(output[:, -1, :])
        
        return out

## Testing

In [ ]:
#Instantiating our first model as an object of the class
RNN_Model_Final = RNN(rnn_hyperparams)
#
#Instantiating our second model as an object of the class
CNN_Model_Final = CNN(cnn_hyperparams)
#
#Instantiating our third model as an object of the class
LSTM_Model_FInal = LSTM(lstm_hyperparams)
#

In [ ]:
criterion1 = nn.MSELoss(reduction = 'mean')
optimizer1 = torch.optim.SGD(RNN_Model_1.parameters(), lr=rnn_hyperparams['learning_rate'], momentum = rnn_hyperparams['momentum'])

In [ ]:
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV
from skorch.callbacks import EarlyStopping, EpochScoring

In [ ]:
net = NeuralNetRegressor(
    RNN_Model_1,
    criterion=nn.MSELoss,
    optimizer=optim.SGD,
    optimizer__momentum=rnn_hyperparams['momentum'],
    lr=rnn_hyperparams['learning_rate'],
    batch_size=1000,
    max_epochs=rnn_hyperparams['num_epochs'],
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

In [ ]:
net.fit(X_train_t_1, y_train_t_1)

In [ ]:
mse = np.mean((y_pred_1 - y_test_1.to_numpy())**2)
print("Mean Squared Error:", mse)